In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import sqlite3
import struct
from scipy.stats.mstats import gmean

In [2]:
res_folder = Path('.')/'sweeps'/'machsuite'
benchmarks = [
    'aes_aes', 
    'bfs_bulk', 
    'bfs_queue', 
    'fft_strided', 
    'fft_transpose', 
    'gemm_ncubed', 
    'kmp_kmp', 
    'md_grid', 
    'md_knn', 
    'nw_nw', 
    'sort_merge', 
    'sort_radix', 
    'spmv_crs', 
    'spmv_ellpack', 
    # 'stencil_stencil_2d', 
    # 'stencil_stencil_3d', 
    # 'viterbi_viterbi',
]
lats = [10, 20, 30, 40, 50, 60, 70, 80]
lats = np.array(list(zip(lats, range(len(lats)), range(len(lats)))))
lats[:,1] = lats[:,1] * 2
lats[:,2] = lats[:,2] * 2 + 1
lats_studied = lats[:4][:,0].copy()
stats = {
    'system.cpu.numCycles': 'cycles',
    'system.cpu.committedInsts': 'instructions',
    'system.cpu.ipc': 'ipc',
    'system.ruby.network.msg_byte.Control': 'msg_byte_control', 
    'system.ruby.network.msg_byte.Request_Control': 'msg_byte_request_control',
    'system.ruby.network.msg_byte.Response_Control': 'msg_byte_response_control',
    'system.ruby.network.msg_byte.Response_Data': 'msg_byte_response_data',
    'system.mem_ctrls.rank0.totalEnergy': 'mc_energy0',
    'system.mem_ctrls.rank1.totalEnergy': 'mc_energy1',
}

In [3]:
lats

array([[10,  0,  1],
       [20,  2,  3],
       [30,  4,  5],
       [40,  6,  7],
       [50,  8,  9],
       [60, 10, 11],
       [70, 12, 13],
       [80, 14, 15]])

In [4]:
def get_stats(path: Path) -> pd.DataFrame:
    sqlite_file = path
    conn = sqlite3.connect(sqlite_file)
    c_map = conn.cursor()
    c_map.execute("SELECT id,name FROM stats")
    rows = c_map.fetchall()
    conn.close()
    stat_map_df = pd.DataFrame(rows, columns=['id', 'name'])
    stat_map_df = stat_map_df.set_index('id')['name']

    conn = sqlite3.connect(sqlite_file)
    c_scalar = conn.cursor()
    c_scalar.execute("SELECT id,dump,value FROM scalarValue")
    rows = c_scalar.fetchall()
    conn.close()
    df_scalar = pd.DataFrame(rows, columns=['id', 'dump', 'value'])
    df_scalar = df_scalar[df_scalar['dump'] == 0]
    df_scalar['id'] = df_scalar['id'].apply(lambda x: stat_map_df[x] if x in stat_map_df else None)
    df_scalar = df_scalar.dropna()
    df_scalar = df_scalar[df_scalar['id'].apply(lambda x: x in stats.keys())]
    df_scalar['id'] = df_scalar['id'].apply(lambda x: stats[x])
    df_scalar = df_scalar.set_index('id')['value']

    conn = sqlite3.connect(sqlite_file)
    c_vector = conn.cursor()
    c_vector.execute("SELECT id,dump,value FROM vectorValue")
    rows = c_vector.fetchall()
    conn.close()
    df_vector = pd.DataFrame(rows, columns=['id', 'dump', 'value'])
    df_vector = df_vector[df_vector['dump'] == 0]
    df_vector['id'] = df_vector['id'].apply(lambda x: stat_map_df[x] if x in stat_map_df else None)
    df_vector = df_vector.dropna()
    df_vector = df_vector[df_vector['id'].apply(lambda x: x in stats.keys())]
    df_vector['id'] = df_vector['id'].apply(lambda x: stats[x])
    df_vector['value'] = df_vector['value'].apply(lambda x: struct.unpack('d', x)[0])
    df_vector = df_vector.set_index('id')['value']

    df = pd.concat([df_scalar, df_vector])
    return df

def get_cycle_table(second=False):
    cycle_table = pd.DataFrame()
    for benchmark in benchmarks:
        for lat, id_s, id_a in lats:
            id = id_a if second else id_s
            path = res_folder/benchmark/str(id)/'outputs'/'stats.db'
            try:
                df = get_stats(path)
            except sqlite3.OperationalError:
                cycle_table.loc[benchmark, lat] = np.nan
                continue
            cycles = df['cycles']
            cycle_table.loc[benchmark, lat] = cycles
    return cycle_table

def get_traffic_table(second=False):
    traffic_table = pd.DataFrame()
    for benchmark in benchmarks:
        for lat, id_s, id_a in lats:
            id = id_a if second else id_s
            path = res_folder/benchmark/str(id)/'outputs'/'stats.db'
            try:
                df = get_stats(path)
            except sqlite3.OperationalError:
                traffic_table.loc[benchmark, lat] = np.nan
                continue
            traffic = df['msg_byte_control'] + df['msg_byte_request_control'] + df['msg_byte_response_control'] + df['msg_byte_response_data']
            traffic_table.loc[benchmark, lat] = traffic
    return traffic_table

def get_interconnect_energy_table(second=False):
    pj_per_bit = 0.3 if second else 1.25
    return get_traffic_table(second=second) * 8 * pj_per_bit * 1e-12

def get_acc_energy_table(second=False):
    acc_energy_table = pd.DataFrame()
    for benchmark in benchmarks:
        for lat, id_s, id_a in lats:
            id = id_a if second else id_s
            path = res_folder/benchmark/str(id)/'outputs'/(benchmark+'_summary')
            with open(path, 'r') as f:
                for line in f:
                    if line.startswith('Cycle'):
                        acc_cycles = float(line.split(':')[-1].split('cycles')[0].strip())
                        acc_energy_table.loc[benchmark, lat] = acc_cycles
                    if line.startswith('Avg Power'):
                        acc_power = float(line.split(':')[-1].split('mW')[0].strip())
                        acc_energy_table.loc[benchmark, lat] *= acc_power
    return acc_energy_table * 1e-12

def get_mc_energy_table(second=False):
    mc_energy_table = pd.DataFrame()
    for benchmark in benchmarks:
        for lat, id_s, id_a in lats:
            id = id_a if second else id_s
            path = res_folder/benchmark/str(id)/'outputs'/'stats.db'
            try:
                df = get_stats(path)
            except sqlite3.OperationalError:
                mc_energy_table.loc[benchmark, lat] = np.nan
                continue
            mc_energy = df['mc_energy0'] + df['mc_energy1']
            mc_energy_table.loc[benchmark, lat] = mc_energy
    return mc_energy_table * 1e-12

In [5]:

import plotly.express as px
import plotly.graph_objs as go
import plotly
plotly.io.templates.default = "plotly_white"


def format_fig(fig, top_margin=0, tickangle=50, leg_x=0.00, leg_y=1.00, big=False):
    fig.update_layout(
        legend=dict(
            yanchor="top",
            y=leg_y,
            xanchor="left",
            x=leg_x,
            orientation='v', 
            bgcolor="White",
            bordercolor="Grey",
            borderwidth=1
        ), 
        font=dict(
            family="Arial",
            size=16, 
            color='Black', 
        ), 
        margin=go.layout.Margin(
            l=0, #left margin
            r=0, #right margin
            b=0, #bottom margin
            t=top_margin  #top margin
        ), 
    )
    fig.update_xaxes(tickangle=tickangle)

In [6]:
cycles_all = get_cycle_table()
cycles = pd.DataFrame(index=cycles_all.index, columns=lats_studied)
for lat in lats_studied:
    cycles[lat] = cycles_all[lat*2] / cycles_all[lat]
cycles.index.name = ' '
cycles.columns.name = 'Interconnect<br>latency'
cycles = cycles.rename(columns=lambda x: str(x)+'ns')
cycles.loc['gmean'] = cycles.apply(lambda x: gmean(x), axis=0)
cycles = cycles * 100 - 100
cycles = cycles.round(2)
cycles;


In [13]:
fig = px.bar(cycles, barmode='group')
fig.update_yaxes(title_text='Execution time increase, %', range=[-3,58])
format_fig(fig, big=True, leg_y=.95, leg_x=.87, tickangle=20)
fig.update_layout(width=900, height=350)
fig.show()
# fig.write_image('figs/exec_time_standard.pdf')

In [14]:
cycles_all = get_cycle_table(second=True)
cycles = pd.DataFrame(index=cycles_all.index, columns=lats_studied)
for lat in lats_studied:
    cycles[lat] = cycles_all[lat*2] / cycles_all[lat]
cycles.index.name = ' '
cycles.columns.name = 'Interconnect<br>latency'
cycles = cycles.rename(columns=lambda x: str(x)+'ns')
cycles.loc['gmean'] = cycles.apply(lambda x: gmean(x), axis=0)
cycles = cycles * 100 - 100
cycles = cycles.round(2)
cycles;


In [15]:
fig = px.bar(cycles, barmode='group')
fig.update_yaxes(title_text='Execution time increase, %', range=[-3,65])
format_fig(fig, big=True, leg_y=.95, leg_x=.87, tickangle=20)
fig.update_layout(width=900, height=350)
fig.show()
# fig.write_image('figs/exec_time_advanced.pdf')

In [16]:
traffic_energy_all = get_interconnect_energy_table()
acc_energy_all = get_acc_energy_table()
mc_energy_all = get_mc_energy_table()

energy_before = (traffic_energy_all + acc_energy_all + mc_energy_all)[lats_studied]
energy_after = (2*traffic_energy_all + acc_energy_all + mc_energy_all)[lats_studied*2]
energy_after = energy_after.rename(columns=lambda x:int(x/2))

energy_increase = energy_after / energy_before
energy_increase.index.name = ' '
energy_increase.columns.name = 'Interconnect<br>latency'
energy_increase = energy_increase.rename(columns=lambda x: str(x)+'ns')

energy_increase.loc['gmean'] = energy_increase.apply(lambda x: gmean(x), axis=0)
energy_increase = energy_increase * 100 - 100
energy_increase = energy_increase.round(2)
energy_increase;

In [17]:
fig = px.bar(energy_increase, barmode='group')
fig.update_yaxes(title_text='Execution energy increase, %', range=[-2,55])
format_fig(fig, big=True, leg_y=.95, leg_x=.87, tickangle=20)
fig.update_layout(width=900, height=350)
fig.show()
# fig.write_image('figs/exec_energy_standard.pdf')


In [18]:
traffic_energy_all = get_interconnect_energy_table(second=True)
acc_energy_all = get_acc_energy_table(second=True)
mc_energy_all = get_mc_energy_table(second=True)

energy_before = (traffic_energy_all + acc_energy_all + mc_energy_all)[lats_studied]
energy_after = (2*traffic_energy_all + acc_energy_all + mc_energy_all)[lats_studied*2]
energy_after = energy_after.rename(columns=lambda x:int(x/2))

energy_increase = energy_after / energy_before
energy_increase.index.name = ' '
energy_increase.columns.name = 'Interconnect<br>latency'
energy_increase = energy_increase.rename(columns=lambda x: str(x)+'ns')

energy_increase.loc['gmean'] = energy_increase.apply(lambda x: gmean(x), axis=0)
energy_increase = energy_increase * 100 - 100
energy_increase = energy_increase.round(2)
energy_increase;

FileNotFoundError: [Errno 2] No such file or directory: 'sweeps/machsuite/aes_aes/1/outputs/aes_aes_summary'

In [ ]:
fig = px.bar(energy_increase, barmode='group')
fig.update_yaxes(title_text='Execution energy increase, %', range=[-2,55])
format_fig(fig, big=True, leg_y=.95, leg_x=.87, tickangle=20)
fig.update_layout(width=900, height=350)
fig.show()
# fig.write_image('figs/exec_energy_advanced.pdf')
